In [26]:
import pandas as pd
import morph_kgc as mgc
import os

In [27]:
if os.getcwd().split('/')[-1] == 'rdf_permissions_example':
    os.chdir('src')

os.getcwd()

'/mnt/c/repos/rdf_permissions_example/src'

# Base Graph Load

In [28]:
config = """
[manager_mapping]
mappings = ../vocab/manager_mapping_metadata.ttl

[items_mapping]
mappings = ../vocab/items_mapping_metadata.ttl

[rep_mapping]
mappings = ../vocab/sales_rep_metadata.ttl
"""

In [29]:
g = mgc.materialize(config)

INFO | 2025-04-19 12:20:44,054 | 16 mapping rules retrieved.
INFO | 2025-04-19 12:20:44,069 | Mapping partition with 15 groups generated.
INFO | 2025-04-19 12:20:44,070 | Maximum number of rules within mapping group: 2.
INFO | 2025-04-19 12:20:44,071 | Mappings processed in 0.796 seconds.
INFO | 2025-04-19 12:20:44,794 | Number of triples generated in total: 38217.


In [30]:
# Validate the graph loaded (no inferences as we haven't run a rules engine)
query = """
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix : <local://security.permissions.net/vocab#>
prefix data: <local://security.permissions.net/entities#>

SELECT * WHERE {
    ?x a :Record ;
        :item ?item ;
        :country ?country ;
}
LIMIT 5
      """

query_result = g.query(query)

for row in query_result:
    print(row['x'], row['item'], row['country'])

In [31]:
# Add the ontology
g.parse('../vocab/permissions_ontology.ttl', format='turtle')

# Serialize the RDFLib graph to RDF/XML format
rdf_xml_data = g.serialize(format='xml', destination='../data/server/graph_data.rdf')